In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.svm as svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics 
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score, cross_validate
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier


In [58]:
def get_clf_eval(y_test, y_pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test,pred_proba)

    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))

In [2]:
# dataset 작업요
df = pd.read_csv('scaled_data.csv', index_col=0)
df.head(3)

,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,직업,가족 구성원 수,산업군,나이,가입연수,도시구분,월간 수입
0,0,1,1,1,3,0,0,3,1,0,1,4.0,5,1,2,3,0
1,0,0,1,0,1,1,0,3,1,0,4,2.0,16,2,1,1,3
2,0,1,0,1,1,0,0,3,1,0,5,2.0,16,1,1,1,1


In [3]:
df['TARGET'].value_counts()

TARGET
0    53561
1     6427
Name: count, dtype: int64

In [4]:
# feature와 label 분리
feature = df.drop('TARGET', axis=1)
label = df['TARGET']

In [5]:
# train test 분리
X_train, X_test, y_train , y_test = train_test_split(feature, label, test_size=0.3 , random_state=42)

In [ ]:
# 언더샘플링 - 소수라벨과 다수라벨 비율 1:1

from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority')
X_train, y_train = undersample.fit_resample(X_train, y_train)

In [7]:
# 오버샘플링 - 소수라벨과 다수라벨 비율 1:1 / 오버 샘플링이 좋음 투터님 피셜

from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [ ]:
# SMOTE

from imblearn.over_sampling import SMOTE
smote_sample = SMOTE(sampling_strategy='minority') 
X_train, y_train = smote_sample.fit_resample(X_train, y_train)

In [66]:
# 1. Decision Tree를 이용한 학습

dt = DecisionTreeClassifier(random_state=43)

# 교차검증을 통한 과적합 or 과소적합 여부 확인
cross_val = cross_val_score(dt , X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))

0.9032920818564202


In [67]:
# GridSearchCV를 적용해 Decision Tree의 교차검증 및 하이퍼파라미터 튜닝

parameters = {'max_depth' : [2, 3, 4, 5],
             'min_samples_split' : [1, 3, 5, 7, 9]}

grid_dt = GridSearchCV(dt, param_grid = parameters, cv=3, refit=True)
grid_dt.fit(X_train, y_train)

dt = grid_dt.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_dt.best_params_}")
print(f"최고 예측 정확도: {grid_dt.best_score_:.4f}")

최적 하이퍼 파라미터: {'max_depth': 5, 'min_samples_split': 3}
최고 예측 정확도: 0.5981


In [68]:
# 임곗값 조정을 통한 recall값 향상
dt.fit(X_train, y_train)
pred = dt.predict(X_test)
pred_proba = dt.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test, custom_pred, pred_proba)

오차행렬:
 [[10113  5964]
 [  841  1079]]

정확도: 0.6219
정밀도: 0.1532
재현율: 0.5620
F1: 0.2408
AUC: 0.6245


In [63]:
# 2. RandomForest를 이용한 학습

rf = RandomForestClassifier(random_state=43)

cross_val = cross_val_score(rf , X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))


0.9453767420394422


In [64]:
# 하이퍼파라미터 조정

parameters = {
    'max_depth' : [6, 8, 12],
    'min_samples_split' : [16, 24]
}

grid_rf = GridSearchCV(rf, param_grid = parameters, cv=3, refit=True)
grid_rf.fit(X_train, y_train)

rf = grid_rf.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_rf.best_params_}")
print(f"최고 예측 정확도: {grid_rf.best_score_:.4f}")

최적 하이퍼 파라미터: {'max_depth': 12, 'min_samples_split': 16}
최고 예측 정확도: 0.7454


In [65]:
# 임계값 조정 및 적용

rf.fit(X_train, y_train)
pred = rf.predict(X_test)
pred_proba = rf.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test, custom_pred, pred_proba)


오차행렬:
 [[11721  4356]
 [ 1104   816]]

정확도: 0.6966
정밀도: 0.1578
재현율: 0.4250
F1: 0.2301
AUC: 0.6180


In [50]:
# 3. Logistic regrssion 을 이용한 학습

lr = LogisticRegression(random_state=43)

cross_val = cross_val_score(lr, X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))


0.5849562164179575


In [51]:
parameters = {'penalty': ['l2','l1'],
          'C':[0.01,0.1,1,10]}

grid_lr = GridSearchCV(lr, param_grid = parameters, cv=3, refit=True)
grid_lr.fit(X_train, y_train)

lr = grid_lr.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_lr.best_params_}")
print(f"최고 예측 정확도: {grid_lr.best_score_:.4f}")

최적 하이퍼 파라미터: {'C': 0.01, 'penalty': 'l2'}
최고 예측 정확도: 0.5844


In [53]:
# 임계값 조정 및 모델 적용

lr.fit(X_train, y_train)
pred = lr.predict(X_test)
pred_proba = lr.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)

Logistic regrssion 정확도 : 0.588
Logistic regrssion 정밀도 : 0.148
Logistic regrssion 재현율 : 0.600
Logistic regrssion f1_score : 0.237
Logistic regrssion roc_auc : 0.593


In [54]:
# 4. KNN 을 이용한 학습

knn = KNeighborsClassifier()
# knn에는 무작위성 추출(random_state)를 하지 않음.

cross_val = cross_val_score(knn , X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))


0.8663429666315047


In [55]:
# 하이퍼파라미터 조정

parameters = {'n_neighbors': [3, 5, 7, 9],
              'weights': ['uniform', 'distance']
          }

grid_knn = GridSearchCV(knn, param_grid = parameters, cv=3, refit=True)
grid_knn.fit(X_train, y_train)

knn = grid_knn.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_knn.best_params_}")
print(f"최고 예측 정확도: {grid_knn.best_score_:.4f}")

최적 하이퍼 파라미터: {'n_neighbors': 3, 'weights': 'distance'}
최고 예측 정확도: 0.8822


In [57]:
# 임계값 조정 및 모델 적용

knn.fit(X_train, y_train)
pred = knn.predict(X_test)
pred_proba = knn.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)

KNN 정확도 : 0.810
KNN 정밀도 : 0.130
KNN 재현율 : 0.139
KNN f1_score : 0.134
KNN roc_auc : 0.514


In [ ]:
# 5. Lightgbm을 이용한 학습

lgbm= LGBMClassifier(random_state=43)

cross_val = cross_val_score(lgbm, X_train , y=y_train ,cv=5, scoring='accuracy')
print(np.mean(cross_val))

In [ ]:
# 하이퍼파라미터 조정

parameters = {'n_neighbors': [100, 500],
              'learning_rate': [0.05, 0.1]
          }

grid_lgbm = GridSearchCV(lgbm, param_grid = parameters, cv=2, verbose=1, refit=True)
grid_lgbm.fit(X_train, y_train)

lgbm = grid_lgbm.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_lgbm.best_params_}")
print(f"최고 예측 정확도: {grid_lgbm.best_score_:.4f}")

In [ ]:
# 임계값 조정 및 모델 적용

lgbm.fit(X_train, y_train)
pred = lgbm.predict(X_test)
pred_proba = lgbm.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)


In [70]:
# 6. XGboosg 
xgb = XGBClassifier(random_state=43)

cross_val = cross_val_score(xgb, X_train, y=y_train, cv=5, scoring='accuracy')
print(np.mean(cross_val))

0.7239488605420157


In [71]:
# 하이퍼파라미터 조정
parameters = {'learning_rat': [0.01, 0.1],  
              'n_estimators': [50, 100, 200]
          }

grid_xgb = GridSearchCV(xgb, param_grid=parameters, cv=2, verbose=1, refit=True)
grid_xgb.fit(X_train, y_train)

xgb = grid_xgb.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_xgb.best_params_}")
print(f"최고 예측 정확도: {grid_xgb.best_score_:.4f}")

Fitting 2 folds for each of 6 candidates, totalling 12 fits
최적 하이퍼 파라미터: {'learning_rat': 0.01, 'n_estimators': 200}
최고 예측 정확도: 0.7568


In [73]:
# 임계값 조정 및 모델 적용

xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
pred_proba = xgb.predict_proba(X_test)[:, 1].reshape(-1, 1)

binarizer = Binarizer(threshold=0.5).fit(pred_proba)
custom_pred = binarizer.transform(pred_proba)

get_clf_eval(y_test , custom_pred, pred_proba)

오차행렬:
 [[11932  4145]
 [ 1193   727]]

정확도: 0.7034
정밀도: 0.1492
재현율: 0.3786
F1: 0.2141
AUC: 0.5802
